## Crawling sites with search

In [1]:
"""
key imports for the project
"""
import re
from bs4 import BeautifulSoup
import requests

In [2]:
class Content:
    """
    common base class for most pages
    """
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.url = url
        self.title = title
        self.body = body
        
    def print(self):
        """
        printing the content
        """
        print("TOPIC: {}".format(self.topic))
        print("URL: {}".format(self.url))
        print("TITLE: {}".format(self.title))
        print("BODY: \n{}".format(self.body))
                 

In [3]:
class Website:
    """
    contain info about the website structure
    """
    def __init__(self, name, url, searchUrl, resultListing, absoluteUrl, resultUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.absoluteUrl = absoluteUrl
        self.resultUrl = resultUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag
        

In [25]:
class Crawler:
    """
    crawling
    """
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')
    
    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''
    
    def search(self, topic, site):
        """
        search a given website for a given topic and records all pages found
        """
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs['href']
            #check to see wheather the url is a relative or absolute URL
            if (site.absoluteUrl):
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url + url)
            if bs is None:
                print("Something went wrong with the page or URL. Skipping!")
                return
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            if title != '' and body != '':
                cotent = Content(topic, url, title, body)
                content.print()

In [26]:
crawler = Crawler()

sitedata = [
    ['O\'Reilly Media', 'http://oreilly.com', 'https://ssearch.oreilly.com/?q=',
        'article.product-result', 'p.title a', True, 'h1', 'section#product-description'],
    ['Reuters', 'http://reuters.com', 'http://www.reuters.com/search/news?blob=', 'div.search-result-content',
        'h3.search-result-title a', False, 'h1', 'div.StandardArticleBody_body_1gnLA'],
    ['Brookings', 'http://www.brookings.edu', 'https://www.brookings.edu/search/?s=',
        'div.list-content article', 'h4.title a', True, 'h1', 'div.post-body']
]
sites = []
for row in sitedata:
    sites.append(Website(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]))
    
topics = ["python", "data science"]
for topic in topics:
    print("GETTING INFO ABOUT: "+ topic)
    for targetSite in sites:
        crawler.search(topic, targetSite)

GETTING INFO ABOUT: python


AttributeError: 'bool' object has no attribute 'replace'